# How to use Laundromat

#### Import the SpacyModel class. This is the main class that controls Laundromat

In [ ]:
from laundromat.spacy.spacy_model import SpacyModel

#### Instantiate your model. The class takes a path to an existing model as an argument. If no argument is given the class defaults to the nb_core_news_lg SpaCy model.

In [ ]:
nlp = SpacyModel() 

#### Consider the following text

In [ ]:
text = ("Hei, mitt navn er Ola Nordmann. Jeg er 20 år og jobber 50 prosent "
"stilling i Arbeidsgiver AS i Norge. Du kan nå meg på 99999999.")

#### There are a variety of functions that can be applied depending on your needs. Keep in mind that the predict() function returns the start and end position in the form of token index, i.e. which token(s) in the text are part of the entity

In [ ]:
#returns [predicted entity, start position, end position, certainty]
entities = nlp.predict(text)
print(entities)

#### If you want predictions prettier and no return you can use the display function. It highlights the given text with predicted entities.

In [ ]:
nlp.display(text)

#### You can also return a dictionary of entity counts.

In [ ]:
nlp.count(text)

#### If you want to replace the found entities with something, you can use the replace() function. It returns the replaced text. The replacement argument determines which kind of replacement is performed, accepted inputs are entity, character, pad, and shuffle. The default is entity. You can also supply the character to be used for character or pad replacement using the replacement_char argument.

In [ ]:
replaced_text_1 = nlp.replace(text, replacement="entity", replacement_char=":^)")
print(replaced_text_1)

In [ ]:
replaced_text_2 = nlp.replace(text, replacement="pad", replacement_char=":^)")
print(replaced_text_2)

In [ ]:
replaced_text_3 = nlp.replace(text)
print(replaced_text_3)

#### Laundromat also contains the similarity() function which returns the cosine similarity between the unaltered text and the replaced text.

In [ ]:
nlp.similarity(replaced_text_1)

In [ ]:
nlp.similarity(replaced_text_2)

In [ ]:
nlp.similarity(replaced_text_3)

#### The add_patterns() function adds the built-in regex and lookup functionality to the nlp pipeline. Observe how functionality is increased after we call on this function.

In [ ]:
nlp.add_patterns()

In [ ]:
nlp.predict(text)

In [ ]:
nlp.display(text)

#### You can also disable the NER model if you only wish to use regex and lookup.

In [ ]:
nlp.disable_ner()

In [ ]:
nlp.display(text)

#### And enable it again

In [ ]:
nlp.enable_ner()

In [ ]:
nlp.display(text)

In [ ]:
#Laundromat comes with list lookup disabled by default, to enable it
# use add_patterns with the parameter lookup = True
nlp = SpacyModel()
nlp.add_patterns(lookup=True)

#### Now we come to a core functionality of Laundromat, which is the ability to easily add custom RegEx expressions or lookup lists by calling a function.

In [ ]:
#First let us consider RegExes. You can print the current regex_labels by calling
# .print_regex_labels()
nlp.print_regex_labels()

In [ ]:
#Next let us add a regex for detecting all lower case four letter words.
#You can do this by calling the .new_regex() function.
nlp.new_regex(pattern=r"([a-z]{4})", context="Four letter words", label="FLW")

In [ ]:
#Now we can see that it is part of the list of RegExes.
nlp.print_regex_labels()

In [ ]:
#We see that our new regex correctly identifies the two lower case four letter words.
nlp.predict(text)

In [ ]:
#Now to lookup lists. These lookup lists are stored in a list of tuples in the form of 
# (lookup list path, entity label). Only the first column of these lists will be considered.
#To print the current lookup lists use the .print_matcher_lists() function.
nlp.print_matcher_lists()

In [ ]:
#To add a lookup list us the .add_list() function.
nlp.add_list(("Some random", "RANDOM"))

In [ ]:
#As we can see, our new list was added
nlp.print_matcher_lists()

#### Now let us explore how to train a model. We shall train a new entity type, but to strengthen performance on already existing entities one just has to provide data with labels corresponding to existing entities. The following "dataset" was copied from SpaCy's website on training new entities. Beware that certain functionality only works for the entity types defined in the code, and so the following example does not inclu

In [ ]:
#Note that the index values of the labels are given in string index.
#SpaCy operates with both string index and token index, i.e. which token in the text it is,
# which can be confusing at times.
nlp = SpacyModel()
LABEL = "ANIMAL"

TRAIN_DATA = [
    (
        "Horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, LABEL)]},
    ),
    ("Do they bite?", {"entities": []}),
    (
        "horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, LABEL)]},
    ),
    ("horses pretend to care about your feelings", {"entities": [(0, 6, LABEL)]}),
    (
        "they pretend to care about your feelings, those horses",
        {"entities": [(48, 54, LABEL)]},
    ),
    ("horses?", {"entities": [(0, 6, LABEL)]}),
]

In [ ]:
#We see that our model is not capable of recognising horses.
for data in TRAIN_DATA:
    print(nlp.predict(data[0]))

#### Keep in mind that display() only has functionality for existing entities. It will not display new entities.

#### First let us test our performance on the data. Laundromat contains three scoring functions: SpaCy's built in F1 scorer, a confusion matrix function, and a custom scoring function. The latter two allow for a strict or lax scoring rule, and the custom scoring function prints a variety of metrics.

In [ ]:
nlp.f1_scorer(TRAIN_DATA)

In [ ]:
nlp.confusion_matrix(TRAIN_DATA)

In [ ]:
nlp.print_scores(TRAIN_DATA)

#### Now let us train on the available data. The train() function accepts both number of training iterations and output directory as arguments, but these default to 30 and None (which means the model is not saved) respectively.

In [ ]:
nlp.train(TRAIN_DATA, labels=[LABEL], n_iter=35)

In [ ]:
nlp.f1_scorer(TRAIN_DATA)

In [ ]:
nlp.confusion_matrix(TRAIN_DATA)

In [ ]:
nlp.print_scores(TRAIN_DATA)

In [ ]:
for data in TRAIN_DATA:
    print(nlp.predict(data[0]))

#### We observe a strong improvement in  performance. Let us now test it on a new sentence.

In [ ]:
horse_text = "Horses are my best friends."

In [ ]:
nlp.predict(horse_text)